In [1]:
!pip install zemberek-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.1/95.1 MB 6.8 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.8-py3-none-any.whl size=141214 sha256=fea8ebc66c8361206fc135e3a49d30ba784008397d2b59cda27a7b951ccdc492
  Stored in directory: /root/.cache/pip/wheels/a7/20/bd/e1477d664f22d99989fd28ee1a43d6633dddb5cb9e801350d5
Successfully built antlr4-python3-runtime


In [3]:
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from gensim.models import KeyedVectors, Word2Vec
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Input
from google.colab import drive
from zemberek import TurkishMorphology, TurkishSpellChecker

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Define paths
dataset_path = '/content/drive/My Drive/DATASET/final_hate_speech.xlsx'
word2vec_path = '/content/drive/My Drive/DATASET/word2vec_tr.model'
fine_tuned_path = '/content/drive/My Drive/DATASET/word2vec_tr_finetuned.model'

# Load dataset
data = pd.read_excel(dataset_path)

In [5]:
# Initialize Zemberek for spell checking
morphology = TurkishMorphology.create_with_defaults()
spell_checker = TurkishSpellChecker(morphology)

cache = {}

def clean_turkish_text_with_cache(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+', '', text)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = ''.join([char for char in text if char.isalnum() or char.isspace()])  # Keep alphanumeric and spaces
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    corrected_words = []
    for word in text.split():
        if word in cache:
            corrected_words.append(cache[word])
        else:
            suggestions = spell_checker.suggest_for_word(word)
            correction = suggestions[0] if suggestions else word
            cache[word] = correction
            corrected_words.append(correction)
    return ' '.join(corrected_words) if corrected_words else "EMPTY"

# Apply preprocessing with caching
data['tweet_cleaned'] = data['tweet'].apply(clean_turkish_text_with_cache)

# Tokenize cleaned tweets
sentences = [tweet.split() for tweet in data['tweet_cleaned'] if tweet != "EMPTY"]

INFO:zemberek.morphology.turkish_morphology:TurkishMorphology instance initialized in 15.941426753997803


2025-01-01 16:37:29,643 - zemberek.morphology.turkish_morphology - INFO
Msg: TurkishMorphology instance initialized in 15.941426753997803



In [6]:
 # Load pre-trained Word2Vec model
word2vec_model = KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

# Fine-tune Word2Vec
new_model = Word2Vec(vector_size=word2vec_model.vector_size, min_count=1)
new_model.build_vocab(sentences)
new_model.build_vocab([list(word2vec_model.key_to_index.keys())], update=True)
new_model.wv.vectors = np.copy(word2vec_model.vectors)
new_model.train(sentences, total_examples=len(sentences), epochs=10)
new_model.save(fine_tuned_path)


INFO:gensim.models.keyedvectors:loading projection weights from /content/drive/My Drive/DATASET/word2vec_tr.model


2025-01-01 17:56:49,379 - gensim.models.keyedvectors - INFO
Msg: loading projection weights from /content/drive/My Drive/DATASET/word2vec_tr.model



INFO:gensim.utils:KeyedVectors lifecycle event {'msg': 'loaded (412457, 400) matrix of type float32 from /content/drive/My Drive/DATASET/word2vec_tr.model', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-01-01T17:57:13.098185', 'gensim': '4.3.3', 'python': '3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}


2025-01-01 17:57:13,098 - gensim.utils - INFO
Msg: KeyedVectors lifecycle event {'msg': 'loaded (412457, 400) matrix of type float32 from /content/drive/My Drive/DATASET/word2vec_tr.model', 'binary': True, 'encoding': 'utf8', 'datetime': '2025-01-01T17:57:13.098185', 'gensim': '4.3.3', 'python': '3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'load_word2vec_format'}



INFO:gensim.utils:Word2Vec lifecycle event {'params': 'Word2Vec<vocab=0, vector_size=400, alpha=0.025>', 'datetime': '2025-01-01T17:57:13.106225', 'gensim': '4.3.3', 'python': '3.10.12 (main, Nov  6 2024, 20:22:13) [GCC 11.4.0]', 'platform': 'Linux-6.1.85+-x86_64-with-glibc2.35', 'event': 'created'}


...

2025-01-01 17:57:56,918 - gensim.utils - INFO
Msg: storing np array 'syn1neg' to /content/drive/My Drive/DATASET/word2vec_tr_finetuned.model.syn1neg.npy



INFO:gensim.utils:not storing attribute cum_table


2025-01-01 17:58:25,973 - gensim.utils - INFO
Msg: not storing attribute cum_table



INFO:gensim.utils:saved /content/drive/My Drive/DATASET/word2vec_tr_finetuned.model


2025-01-01 17:58:27,235 - gensim.utils - INFO
Msg: saved /content/drive/My Drive/DATASET/word2vec_tr_finetuned.model



In [ ]:
data.head()

In [8]:
def text_to_word2vec(text, model, vector_size=300):
    if not text or text.strip() == "EMPTY":
        return np.zeros(vector_size, dtype=np.float32)
    words = text.split()
    embeddings = [model[word] for word in words if word in model.key_to_index]
    if not embeddings:
        return np.zeros(vector_size, dtype=np.float32)
    return np.mean(embeddings, axis=0)

In [9]:
X = np.array([text_to_word2vec(tweet, new_model.wv, vector_size=new_model.vector_size)
             for tweet in data['tweet_cleaned']])
y = data['etiket'].values
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [10]:
# Define resampling methods
resampling_methods = {
    "Original": None,
    "Oversampling": RandomOverSampler(random_state=42),
    "Undersampling": RandomUnderSampler(random_state=42),
    "Combined": SMOTEENN(random_state=42),
}

# Define ML models
models = {
    "Random Forest": RandomForestClassifier(random_state=42),
    "XGBoost": XGBClassifier(random_state=42, tree_method='hist', use_label_encoder=False),
    "LightGBM": LGBMClassifier(random_state=42),
}

In [11]:
# ANN Model
def build_ann(input_dim):
    model = Sequential([
        Input(shape=(input_dim,)),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(64, activation='relu'),
        Dropout(0.2),
        Dense(len(label_encoder.classes_), activation='softmax'),
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Train and evaluate models
results = []

In [12]:
for res_name, resampler in resampling_methods.items():
    if resampler:
        X_resampled, y_resampled = resampler.fit_resample(X_train, y_train)
    else:
        X_resampled, y_resampled = X_train, y_train

    for model_name, model in models.items():
        model.fit(X_resampled, y_resampled)
        y_pred = model.predict(X_test)
        results.append({
            "Model": model_name,
            "Resampling": res_name,
            "Accuracy": accuracy_score(y_test, y_pred),
            "Precision": precision_score(y_test, y_pred, average='macro'),
            "Recall": recall_score(y_test, y_pred, average='macro'),
            "F1-Score": f1_score(y_test, y_pred, average='macro'),
        })

    # ANN
    ann_model = build_ann(X_resampled.shape[1])
    ann_model.fit(X_resampled, y_resampled, epochs=10, batch_size=32, verbose=0)
    y_pred_ann = np.argmax(ann_model.predict(X_test), axis=1)
    results.append({
        "Model": "ANN",
        "Resampling": res_name,
        "Accuracy": accuracy_score(y_test, y_pred_ann),
        "Precision": precision_score(y_test, y_pred_ann, average='macro'),
        "Recall": recall_score(y_test, y_pred_ann, average='macro'),
        "F1-Score": f1_score(y_test, y_pred_ann, average='macro'),
    })

# Results DataFrame
results_df = pd.DataFrame(results)
print(results_df)

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [17:59:00] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and 

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.051503 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 102000
[LightGBM] [Info] Number of data points in the train set: 8179, number of used features: 400
[LightGBM] [Info] Start training from score -0.277827
[LightGBM] [Info] Start training from score -1.488549
[LightGBM] [Info] Start training from score -4.082071


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:484: FutureWarning: `BaseEstimator._check_n_features` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_n_features` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: FutureWarning: `BaseEstimator._check_feature_names` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation._check_feature_names` instead.
  warnings.warn(
/usr/local/lib

...

64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTEENN or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.valida

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.077581 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 102000
[LightGBM] [Info] Number of data points in the train set: 12421, number of used features: 400
[LightGBM] [Info] Start training from score -3.453334
[LightGBM] [Info] Start training from score -0.755857
[LightGBM] [Info] Start training from score -0.695646


/usr/local/lib/python3.10/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


64/64 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
            Model     Resampling  Accuracy  Precision    Recall  F1-Score
0   Random Forest       Original  0.751589   0.466130  0.344756  0.311199
1         XGBoost       Original  0.773105   0.503852  0.382541  0.378850
2        LightGBM       Original  0.776528   0.510048  0.387303  0.386086
3             ANN       Original  0.775061   0.462812  0.442086  0.448332
4   Random Forest   Oversampling  0.750122   0.478317  0.340867  0.302419
5         XGBoost   Oversampling  0.784352   0.497498  0.408351  0.415861
6        LightGBM   Oversampling  0.772616   0.460907  0.421591  0.429389
7             ANN   Oversampling  0.653790   0.474653  0.482929  0.449586
8   Random Forest  Undersampling  0.478240   0.384148  0.461181  0.341091
9         XGBoost  Undersampling  0.450856   0.398784  0.475290  0.341586
10       LightGBM  Undersampling  0.461125   0.396600  0.486017  0.345190
11            ANN  Undersampling  0.335452   0.390200  0.434073  0.281599